In [1]:
import numpy as np
import pandas as pd
import glob as gb 
import os

### Состояние склада на каждый день

Создание имени файла

In [2]:
def create_filenames(letter, number):
    
    c = chr(ord('0') + number)
    str_i = 'MS-' + letter + c + '-inventory.csv'
    str_sl = 'MS-' + letter + c + '-sell.csv'
    str_spl = 'MS-' + letter + c + '-supply.csv'
    str_d = 'MS-' + letter + c + '-daily.csv'
    return str_i, str_sl, str_spl, str_d

Количество магазинов в штате

In [3]:
def num_shop(state):
    
    if state == 'b' or state == 'm':
        max = 2
    elif state == 's':
        max = 5
    return max

In [13]:
def get_files(state, directory):
    max = num_shop(state)
    files = {}
    path = {}
    shop = 0
    while shop < max:
        c = chr(ord('0') + shop + 1)
        path[shop] = gb.glob(directory + state + c + '*.csv')
        print(path[shop])
        files[shop] = {}
        for j in range(0, len(path[shop])):
            files[shop][j]= pd.read_csv(path[shop][j], parse_dates=['date'], dayfirst=True)
        shop = shop + 1
    return files, max

In [ ]:
def sell_upg(sell):
    
    upg_tmp = pd.DataFrame(columns=['sku_num'], index = sell.index)
    upg_tmp['sku_num'] = sell['sku_num'].apply(lambda x: 0 if x.find('ap') == 6 else 1)
    daily = pd.crosstab(upg_tmp.index, upg_tmp['sku_num'], margins = True)
    daily = daily.drop(['All'])
    daily_new = pd.DataFrame(columns=[ 'apple', 'pen'], 
                             index = upg_tmp.index.drop_duplicates())
    daily_new['apple'] = daily[0].values
    daily_new['pen'] = daily[1].values
        
    return daily_new

### Task 1

In [17]:
sold = {}
def daily_stor(state, id):
    
    files, max = get_files(state, './input/*')
    answer = {}
    
    INV = 0
    SELL = 1
    SPL = 2
    DAILY = 3
    
    shop = 0
    while shop < max:
        
        daily_new = sell_upg(files[shop][SELL])
        
        print(daily_new.head())
        
        daily_new.index = files[shop][SELL]['date'].drop_duplicates()
        #daily_new = daily_new.drop(['date'], axis=1)
        daily_new['apple'] = -daily_new['apple']
        daily_new['pen'] = -daily_new['pen']
        
        files[shop][SPL].index = files[shop][SPL].date
        #files[shop][SPL] = files[shop][SPL].drop(['date'], axis=1)
        
        df = pd.concat([files[shop][SPL], daily_new]).sort_index()
        df = df.resample('D').sum()
        
        l = df.index.size
        df['apple_sum'] = df['apple'].rolling(str(l) + 'D').sum()
        df['pen_sum'] = df['pen'].rolling(str(l) + 'D').sum()
        df = df.drop(['apple', 'pen'], axis=1)
        df = df.rename(columns={'apple_sum': 'apple', 'pen_sum': 'pen'})
        df['date'] = daily_new['date'] 
        print(df.head())
        
        
        name = create_filenames(state, shop + 1)[DAILY]
        df.to_csv('answer/' + name, index=False)
        answer[shop] = df
        shop = shop + 1
        
    return answer
    

### Task 2

In [ ]:
def stolen(sell, supply, inventory):
    
    st = pd.DataFrame(columns=['apple', 'pen'], index = inventory.index)
    sell = sell.resample('M').sum()
    supply = supply.resample('M').sum()
    inv = pd.DataFrame(columns=['apple', 'pen'], index = inventory.index)
    inv['apple'] = inventory['apple'] - inventory['apple'] .shift(1)
    inv.loc[inv.index[0], 'apple'] = inventory.loc[inventory.index[0], 'apple']
    inv['pen'] = inventory['pen'] - inventory['pen'] .shift(1)
    inv.loc[inv.index[0], 'pen'] = inventory.loc[inventory.index[0], 'pen']
    st['apple'] = inv['apple'] + sell['apple'] - supply['apple']
    st['pen'] = inv['pen'] + sell['pen'] - supply['pen'] 
    return st.abs()

In [18]:
daily_b = daily_stor('b', 0)
print(daily_b)
daily_m = daily_stor('m', 1)
print(daily_m)
daily_s = daily_stor('s', 2)
print(daily_s)

['./input\\MS-b1-inventory.csv', './input\\MS-b1-sell.csv', './input\\MS-b1-supply.csv']
['./input\\MS-b2-inventory.csv', './input\\MS-b2-sell.csv', './input\\MS-b2-supply.csv']
sku_num                    0       1      All
date                                         
2006-01-01 00:00:00     1815     156     1971
2006-01-02 00:00:00     1862     143     2005
2006-01-03 00:00:00     1880     171     2051
2006-01-04 00:00:00     1797     153     1950
2006-01-05 00:00:00     1942     133     2075
2006-01-06 00:00:00     1898     149     2047
2006-01-07 00:00:00     1895     138     2033
2006-01-08 00:00:00     1836     120     1956
2006-01-09 00:00:00     1868     125     1993
2006-01-10 00:00:00     1879     156     2035
2006-01-11 00:00:00     1796     151     1947
2006-01-12 00:00:00     1915     140     2055
2006-01-13 00:00:00     1920     136     2056
2006-01-14 00:00:00     1872     161     2033
2006-01-15 00:00:00     1822     144     1966
2006-01-16 00:00:00     1894     160    

['./input\\MS-m2-inventory.csv', './input\\MS-m2-sell.csv', './input\\MS-m2-supply.csv']
sku_num                    0       1      All
date                                         
2006-01-01 00:00:00      813      43      856
2006-01-02 00:00:00      849      45      894
2006-01-03 00:00:00      920      33      953
2006-01-04 00:00:00      874      55      929
2006-01-05 00:00:00      826      48      874
2006-01-06 00:00:00      865      48      913
2006-01-07 00:00:00      837      39      876
2006-01-08 00:00:00      793      44      837
2006-01-09 00:00:00      807      61      868
2006-01-10 00:00:00      856      51      907
2006-01-11 00:00:00      800      48      848
2006-01-12 00:00:00      881      42      923
2006-01-13 00:00:00      884      40      924
2006-01-14 00:00:00      852      54      906
2006-01-15 00:00:00      817      45      862
2006-01-16 00:00:00      866      53      919
2006-01-17 00:00:00      876      44      920
2006-01-18 00:00:00      830      48 

['./input\\MS-s2-inventory.csv', './input\\MS-s2-sell.csv', './input\\MS-s2-supply.csv']
['./input\\MS-s3-inventory.csv', './input\\MS-s3-sell.csv', './input\\MS-s3-supply.csv']
['./input\\MS-s4-inventory.csv', './input\\MS-s4-sell.csv', './input\\MS-s4-supply.csv']
['./input\\MS-s5-inventory.csv', './input\\MS-s5-sell.csv', './input\\MS-s5-supply.csv']
sku_num                   0      1     All
date                                      
2006-01-01 00:00:00      96     11     107
2006-01-02 00:00:00      79      6      85
2006-01-03 00:00:00      83     15      98
2006-01-04 00:00:00      86     14     100
2006-01-05 00:00:00      97     12     109
2006-01-06 00:00:00     102      5     107
2006-01-07 00:00:00      91      8      99
2006-01-08 00:00:00     111      5     116
2006-01-09 00:00:00      98     11     109
2006-01-10 00:00:00     100      6     106
2006-01-11 00:00:00      92      6      98
2006-01-12 00:00:00      84      6      90
2006-01-13 00:00:00      83     13      96

sku_num                   0      1     All
date                                      
2006-01-01 00:00:00      85      9      94
2006-01-02 00:00:00      97     10     107
2006-01-03 00:00:00      96      8     104
2006-01-04 00:00:00     105     12     117
2006-01-05 00:00:00      89      6      95
2006-01-06 00:00:00      93      8     101
2006-01-07 00:00:00      95      9     104
2006-01-08 00:00:00     103     13     116
2006-01-09 00:00:00      97     12     109
2006-01-10 00:00:00      93     12     105
2006-01-11 00:00:00     103     10     113
2006-01-12 00:00:00      85      8      93
2006-01-13 00:00:00      86     11      97
2006-01-14 00:00:00      99     14     113
2006-01-15 00:00:00      82      8      90
2006-01-16 00:00:00     102     14     116
2006-01-17 00:00:00      79     13      92
2006-01-18 00:00:00     110     14     124
2006-01-19 00:00:00      99     11     110
2006-01-20 00:00:00      95      4      99
2006-01-21 00:00:00      92     12     104
2006-01-22 

In [48]:
def year_table():
    states = ['b', 'm', 's']
    res = {}
    t = {}
    for i in range (0, 3):
        t[i] = sold[i].resample('Y').sum()
        stolen[i] = stolen[i].resample('Y').sum()
        t[i].insert(3, 'apple_stolen',stolen[i][1])
        t[i].insert(5, 'pen_stolen',stolen[i][2])
        t[i].insert(1, 'state', states[i])
        
        
    res = pd.concat([t[0], t[1], t[2]]).sort_index()

In [16]:
#def check_res():
states = ['b', 'm', 's']
for state in states:
    right_ansm, maxou = get_files(state, './output/*')
    our_ans, maxa = get_files(state, './answer/*') 
    for i in range (0, maxou):
        if (right_ansm[i] != our_ans[i]):
            print('ответы не сошлись')

['./output\\MS-b1-daily.csv', './output\\MS-b1-steal.csv']
['./output\\MS-b2-daily.csv', './output\\MS-b2-steal.csv']
['./answer\\MS-b1-daily.csv']


ValueError: 'date' is not in list